# Merging Data with Pandas

## Introduction

In real-world data analysis, information is rarely stored in a single monolithic table. Instead, data is typically distributed across multiple sources, each representing different aspects, entities, or time periods of the system under study. Combining these disparate tables - **merging** - is a foundational operation in data wrangling, much like performing SQL joins in a relational database. Effective merging allows you to enrich datasets, assemble complete records, and unlock the analytical power of relational thinking.

## The Essence of Merging

### What is a Merge (Join)?

A **merge** combines two or more tables (DataFrames) based on the values of one or more shared columns, called **keys**. This process is analogous to SQL joins, enabling you to bring together related information from separate sources.

### The Most Common Join: **Inner Join**

**Intuition:**  
An *inner join* retrieves only the rows that have matching key values in **both** DataFrames. It acts as a filter - retaining only those records for which the relationship exists in all datasets involved.

- If you imagine each table as a circle in a Venn diagram, an inner join corresponds to the overlapping region: only values present in both circles are included in the result.
- This is the default behaviour in Pandas' `merge` function.

**When to use:**  
- When you want to analyse only the “shared universe” of your datasets - i.e., where data is available in all sources.

### Syntax: `pd.merge()`

The principal function for table merging in Pandas is `pd.merge()`. The basic usage is:

```python
import pandas as pd

merged_df = pd.merge(
    left=table1,
    right=table2,
    on="key_column"        # or on=["key1", "key2"] for composite keys
)
```
- By default, this performs an **inner join** on the specified key(s).


### Example: Inner Join with a Common Key

Suppose you have two DataFrames, each with a `"user_id"` column representing the entity to join on:

```python
merged_df = pd.merge(left=users, right=purchases, on="user_id")
```
- Only users with purchase records - and vice versa - are included in the result.

### Handling Overlapping Column Names: The `suffixes` Argument

When both tables have columns with the same name (apart from the join key), Pandas automatically appends suffixes to distinguish them. You can customise these using the `suffixes` argument:

```python
merged_df = pd.merge(
    left=table1,
    right=table2,
    on="id",
    suffixes=("_left", "_right")
)
```
- This ensures the resulting DataFrame remains unambiguous and self-explanatory.

## Best Practices

- **Explicit key columns:** Always specify the `on=` argument to avoid subtle merge errors, especially when column names are similar but not identical.
- **Review join types:** Inner joins are safe and conservative, but for broader analyses, consider left, right, or outer joins (see Pandas documentation).
- **Inspect results:** Always check the shape and sample rows of your merged DataFrame (`.shape`, `.head()`) to ensure correctness.
- **Column name hygiene:** Use `suffixes` to manage naming collisions for maximum interpretability.



In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns 
import os 

In [2]:
wards = pd.read_pickle("data/ward.p")
census = pd.read_pickle("data/census.p")

In [3]:
wards.head()

,ward,alderman,address,zip
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609
3,4,William D. Burns,"435 EAST 35TH STREET, 1ST FLOOR",60616
4,5,Leslie A. Hairston,2325 EAST 71ST STREET,60649


In [4]:
census.head()

,ward,pop_2000,pop_2010,change,address,zip
0,1,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637


In [5]:
wards_census = wards.merge(census, on="ward")
display(wards_census.head())

,ward,alderman,address_x,zip_x,pop_2000,pop_2010,change,address_y,zip_y
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,William D. Burns,"435 EAST 35TH STREET, 1ST FLOOR",60616,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,Leslie A. Hairston,2325 EAST 71ST STREET,60649,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637


In [6]:
print(wards_census.columns)

Index(['ward', 'alderman', 'address_x', 'zip_x', 'pop_2000', 'pop_2010',
       'change', 'address_y', 'zip_y'],
      dtype='object')


In [7]:
wards_census = wards.merge(census, on="ward", suffixes=("_ward", "_cen"))
display(wards_census.head())

,ward,alderman,address_ward,zip_ward,pop_2000,pop_2010,change,address_cen,zip_cen
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,William D. Burns,"435 EAST 35TH STREET, 1ST FLOOR",60616,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,Leslie A. Hairston,2325 EAST 71ST STREET,60649,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637


In [8]:
print(wards_census.shape)

(50, 9)


### What column to merge on?
Chicago provides a list of taxicab owners and vehicles licensed to operate within the city, for public safety. Your goal is to merge two tables together. One table is called `taxi_owners`, with info about the taxi cab company owners, and one is called `taxi_veh`, with info about each taxi cab vehicle. Both the `taxi_owners` and `taxi_veh` tables have been loaded for you to explore.

Choose the column you would use to merge the two tables on using the `.merge()` method.

In [9]:
# Load the datasets taxi_owners and taxi_veh
taxi_veh = pd.read_pickle("data/taxi_vehicles.p")
taxi_owners = pd.read_pickle("data/taxi_owners.p")

In [10]:
taxi_veh.head()

,vid,make,model,year,fuel_type,owner
0,2767,TOYOTA,CAMRY,2013,HYBRID,SEYED M. BADRI
1,1411,TOYOTA,RAV4,2017,HYBRID,DESZY CORP.
2,6500,NISSAN,SENTRA,2019,GASOLINE,AGAPH CAB CORP
3,2746,TOYOTA,CAMRY,2013,HYBRID,"MIDWEST CAB CO, INC"
4,5922,TOYOTA,CAMRY,2013,HYBRID,SUMETTI CAB CO


In [11]:
taxi_owners.head()

,rid,vid,owner,address,zip
0,T6285,6285,AGEAN TAXI LLC,4536 N. ELSTON AVE.,60630
1,T4862,4862,MANGIB CORP.,5717 N. WASHTENAW AVE.,60659
2,T1495,1495,"FUNRIDE, INC.",3351 W. ADDISON ST.,60618
3,T4231,4231,ALQUSH CORP.,6611 N. CAMPBELL AVE.,60645
4,T5971,5971,EUNIFFORD INC.,3351 W. ADDISON ST.,60618


### Your first inner join
You have been tasked with figuring out what the most popular types of fuel used in Chicago taxis are. To complete the analysis, you need to merge the `taxi_owners` and `taxi_veh` tables together on the vid column. You can then use the merged table along with the `.value_counts()` method to find the most common fuel_type.

In [12]:
# Merge taxi_owners with taxi_veh on the column vid, and save the result to taxi_own_veh.
# Set the left and right table suffixes for overlapping columns of the merge to _own and _veh, respectively.
taxi_own_veh = taxi_owners.merge(taxi_veh, on="vid", suffixes=("_own","_veh"))
print(taxi_own_veh.columns)

Index(['rid', 'vid', 'owner_own', 'address', 'zip', 'make', 'model', 'year',
       'fuel_type', 'owner_veh'],
      dtype='object')


In [13]:
# Select the fuel_type column from taxi_own_veh and print the value_counts() to find the most popular fuel_types used.
print(taxi_own_veh["fuel_type"].value_counts)

<bound method IndexOpsMixin.value_counts of 0         HYBRID
1       GASOLINE
2       GASOLINE
3         HYBRID
4       GASOLINE
          ...   
3514      HYBRID
3515      HYBRID
3516      HYBRID
3517      HYBRID
3518      HYBRID
Name: fuel_type, Length: 3519, dtype: object>


### Inner joins and number of rows returned
All of the merges you have studied to this point are called inner joins. It is necessary to understand that inner joins only return the rows with matching values in both tables. 

In [14]:
# Merge wards and census on the ward column and save the result to wards_census.
ward_census = wards.merge(census, on="ward")
ward_census


,ward,alderman,address_x,zip_x,pop_2000,pop_2010,change,address_y,zip_y
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,William D. Burns,"435 EAST 35TH STREET, 1ST FLOOR",60616,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,Leslie A. Hairston,2325 EAST 71ST STREET,60649,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637
5,6,Roderick T. Sawyer,8001 S. MARTIN LUTHER KING DRIVE,60619,54989,52341,-5%,150 WEST 74TH STREET,60636
6,7,Gregory I. Mitchell,2249 EAST 95TH STREET,60617,54593,51581,-6%,8549 SOUTH OGLESBY AVENUE,60617
7,8,Michelle A. Harris,8539 SOUTH COTTAGE GROVE AVENUE,60619,54039,51687,-4%,1346-1352 EAST 75TH STREET,60649
8,9,Anthony A. Beale,34 EAST 112TH PLACE,60628,52008,51519,-1%,11039-11059 SOUTH WENTWORTH AVENUE,60628
9,10,Susan Sadlowski Garza,10500 SOUTH EWING AVENUE,60617,56613,51535,-9%,10534 SOUTH AVENUE F,46394


# Merging Multiple DataFrames

## Introduction

In practical data analysis, you frequently encounter scenarios where data is distributed across more than two tables—perhaps different files for different years, separate tables for related entities, or multiple sources tracking overlapping phenomena. The ability to **merge multiple DataFrames** is essential for building a unified, coherent dataset from fragmented sources. This task, while conceptually an extension of pairwise merging, introduces new technical and organisational considerations.

This section presents a rigorous and pedagogical approach to merging more than two DataFrames in Pandas, emphasising underlying principles, technical best practices, and clear strategies for complex real-world workflows.

## Why Merge Multiple Tables?

- **Normalised data:** Well-structured datasets are often “normalised,” with each table focusing on a single entity or relationship (e.g., users, transactions, products).
- **Incremental enrichment:** You may need to successively enrich a “core” dataset by merging in supplementary information (demographics, geography, transactions, etc.).
- **Time series and partitioned data:** Data split by year, month, or region often arrives as multiple files/tables requiring consolidation.
- **Integrating data from multiple systems:** Merging is required to assemble a holistic view from various sources or platforms.

## Principles and Theory: Merging Beyond Two Tables

Theoretically, a **multi-way merge** can be decomposed into a sequence of binary (pairwise) merges. The result of each merge serves as the left table in the next merge, and so on.  
However, practical complexities arise:
- **Consistent join keys:** All merges must use compatible keys; mismatched keys lead to data loss or duplication.
- **Naming collisions:** Overlapping column names across tables require explicit disambiguation via `suffixes`.
- **Data loss:** Each inner join restricts the dataset to rows common to all merged tables; outer joins preserve more data but may introduce missing values.

## Single Merge vs. Multiple Merges

### Single (Pairwise) Merge

```python
merged = pd.merge(table1, table2, on="id")
```
- Simple, clear, and effective when you only need to combine two tables.

### Merging Multiple Tables: Stepwise Approach

To merge three or more tables, you typically proceed recursively:

```python
merged = pd.merge(table1, table2, on="id", suffixes=("_t1", "_t2"))
merged = pd.merge(merged, table3, on="id", suffixes=("", "_t3"))
```

Each call combines the result of the previous merge with the next table, one at a time.

### Example: Merging Multiple Tables with Composite Keys and Suffixes

Suppose you have several tables sharing composite keys (multiple columns), and some columns overlap in name:

```python
merged = pd.merge(
    table1,
    table2,
    on=["user_id", "date"],
    suffixes=("_t1", "_t2")
)
merged = pd.merge(
    merged,
    table3,
    on=["user_id", "date"],
    suffixes=("", "_t3")
)
```
- **Composite keys:** Use `on=[...]` to specify all join columns.
- **Suffixes:** Avoid naming collisions as each table is added.

## Chaining `.merge()` for Multiple DataFrames

When merging more than two DataFrames that all share the same join key(s), Pandas allows you to **chain** `.merge()` calls. This produces concise, readable code and maintains clarity about the order and logic of each join.

### Example: Merging Three or More Tables

#### Merging Three Tables

```python
merged = df1.merge(df2, on="col") \
            .merge(df3, on="col")
```
- The result of the first merge (`df1` with `df2`) is merged immediately with `df3` on the same key.

#### Merging Four Tables

```python
merged = df1.merge(df2, on="col") \
            .merge(df3, on="col") \
            .merge(df4, on="col")
```
- The process is associative and left-to-right:  
  `(((df1 + df2) + df3) + df4)`

#### Merging Five or More Tables

```python
merged = df1.merge(df2, on="col") \
            .merge(df3, on="col") \
            .merge(df4, on="col") \
            .merge(df5, on="col")
```
- Each additional table is merged into the accumulating result.

### Key Points and Best Practices

- **Consistent join key:** All tables must share the same key(s). For composite keys, use `on=["col1", "col2"]`.
- **Suffixes for name collisions:** If non-key columns have the same names, manage them with `suffixes` in each call to avoid ambiguity.
- **Join type:** By default, `.merge()` performs an inner join. Specify `how="left"` (or `"outer"`, etc.) if needed for your logic.
- **Readability:** For many tables or programmatic merges, consider using a loop or `functools.reduce()` for even greater generality and maintainability.

### Programmatic Merging with a Loop

If you have a list of DataFrames to merge:

```python
from functools import reduce

merged = reduce(lambda left, right: left.merge(right, on="col"), [df1, df2, df3, df4, df5])
```
- This pattern is particularly useful when the number of tables is not known in advance (e.g., reading from a directory of CSV files).


### Advanced: Different Join Keys and Suffixes

If each merge uses different keys or needs custom suffixes, chain explicitly:

```python
merged = df1.merge(df2, on="key1", suffixes=("_a", "_b")) \
            .merge(df3, on="key2", suffixes=("", "_c")) \
            .merge(df4, on="key3", suffixes=("", "_d"))
```



In [36]:
licenses = pd.read_pickle("data/licenses.p")
biz_owners = pd.read_pickle("data/business_owners.p")

In [18]:
display(licenses.head())
display(licenses.shape)

,account,ward,aid,business,address,zip
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616
1,10,10,829,HONEYBEERS,13200 S HOUSTON AVE,60633
2,10002,14,775,CELINA DELI,5089 S ARCHER AVE,60632
3,10005,12,NaN,KRAFT FOODS NORTH AMERICA,2005 W 43RD ST,60609
4,10044,44,638,NEYBOUR'S TAVERN & GRILLE,3651 N SOUTHPORT AVE,60613


(10000, 6)

In [21]:
display(biz_owners.head())
display(biz_owners.shape)

,account,first_name,last_name,title
0,10,PEARL,SHERMAN,PRESIDENT
1,10,PEARL,SHERMAN,SECRETARY
2,10002,WALTER,MROZEK,PARTNER
3,10002,CELINA,BYRDAK,PARTNER
4,10005,IRENE,ROSENFELD,PRESIDENT


(21352, 4)

In [19]:
ward_licenses = wards.merge(licenses, on="ward", suffixes=("_ward", "_lic"))
display(ward_licenses.head())

,ward,alderman,address_ward,zip_ward,account,aid,business,address_lic,zip_lic
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,12024,NaN,DIGILOG ELECTRONICS,1038 N ASHLAND AVE,60622
1,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,14446,743,EMPTY BOTTLE INC,1035 N WESTERN AVE 1ST,60622
2,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,14624,775,LITTLE MEL'S HOT DOG,2205 N CALIFORNIA AVE,60647
3,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,14987,NaN,MR. BROWN'S LOUNGE,2301 W CHICAGO AVE 1ST,60622
4,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,15642,814,Beat Kitchen,2000-2100 W DIVISION ST,60622


### One-to-many merge
A business may have one or multiple owners. In this exercise, you will continue to gain experience with one-to-many merges by merging a table of business owners, called `biz_owners`, to the `licenses` table. Recall from the video lesson, with a one-to-many relationship, a row in the left table may be repeated if it is related to multiple rows in the right table. In this lesson, you will explore this further by finding out what is the most common business owner title. (i.e., secretary, CEO, or vice president)

In [22]:
# Starting with the licenses table on the left, merge it to the biz_owners table on the column account, and save the results to a variable named licenses_owners.
licenses_owners = licenses.merge(biz_owners, on="account")

In [24]:
# Group the licenses_owners dataframe by the 'title' column to analyze business owner titles
# Use the agg() function with a dictionary to specify aggregation operations for specific columns
# The dictionary format is: {'column_name': 'aggregation_function'}
counted_df = licenses_owners.groupby("title").agg({"account":"count"})
counted_df

,account
title,
ASST. SECRETARY,111
BENEFICIARY,4
CEO,110
DIRECTOR,146
EXECUTIVE DIRECTOR,10
GENERAL PARTNER,21
INDIVIDUAL,268
LIMITED PARTNER,26
MANAGER,134


In [27]:
# Sort counted_df by the number of accounts in descending order, and save this as a variable named sorted_df.
sorted_df = counted_df.sort_values(by="account", ascending=False)
display(sorted_df.head())

,account
title,
PRESIDENT,6259
SECRETARY,5205
SOLE PROPRIETOR,1658
OTHER,1200
VICE PRESIDENT,970


### Total riders in a month
Your goal is to find the total number of rides provided to passengers passing through the Wilson station (`station_name == 'Wilson'`) when riding Chicago's public transportation system on weekdays (`day_type == 'Weekday'`) in July (`month == 7`). Luckily, Chicago provides this detailed data, but it is in three different tables. You will work on merging these tables together to answer the question. This data is different from the business related data you have seen so far, but all the information you need to answer the question is provided.

In [38]:
cal = pd.read_pickle("data/cta_calendar.p")
ridership = pd.read_pickle("data/cta_ridership.p")
stations = pd.read_pickle("data/stations.p")

In [39]:
cal.head()

,year,month,day,day_type
0,2019,1,1,Sunday/Holiday
1,2019,1,2,Weekday
2,2019,1,3,Weekday
3,2019,1,4,Weekday
4,2019,1,5,Saturday


In [40]:
ridership.head()

,station_id,year,month,day,rides
0,40010,2019,1,1,576
1,40010,2019,1,2,1457
2,40010,2019,1,3,1543
3,40010,2019,1,4,1621
4,40010,2019,1,5,719


In [41]:
stations.head()

,station_id,station_name,location
0,40010,Austin-Forest Park,"(41.870851, -87.776812)"
1,40020,Harlem-Lake,"(41.886848, -87.803176)"
2,40030,Pulaski-Lake,"(41.885412, -87.725404)"
3,40040,Quincy/Wells,"(41.878723, -87.63374)"
4,40050,Davis,"(42.04771, -87.683543)"


In [42]:
# Merge the ridership and cal tables together, starting with the ridership table on the left and save the result to the variable ridership_cal. If you code takes too long to run, your merge conditions might be incorrect.
ridership_cal = ridership.merge(cal, on=["year", "month", "day"])
ridership_cal

,station_id,year,month,day,rides,day_type
0,40010,2019,1,1,576,Sunday/Holiday
1,40010,2019,1,2,1457,Weekday
2,40010,2019,1,3,1543,Weekday
3,40010,2019,1,4,1621,Weekday
4,40010,2019,1,5,719,Saturday
...,...,...,...,...,...,...
3280,41660,2019,12,27,13898,Weekday
3281,41660,2019,12,28,9485,Saturday
3282,41660,2019,12,29,7581,Sunday/Holiday
3283,41660,2019,12,30,15332,Weekday


In [46]:
# Extend the previous merge to three tables by also merging the stations table.
ridership_cal_stations = ridership.merge(cal, on=['year','month','day']).merge(stations, on="station_id")
ridership_cal_stations

,station_id,year,month,day,rides,day_type,station_name,location
0,40010,2019,1,1,576,Sunday/Holiday,Austin-Forest Park,"(41.870851, -87.776812)"
1,40010,2019,1,2,1457,Weekday,Austin-Forest Park,"(41.870851, -87.776812)"
2,40010,2019,1,3,1543,Weekday,Austin-Forest Park,"(41.870851, -87.776812)"
3,40010,2019,1,4,1621,Weekday,Austin-Forest Park,"(41.870851, -87.776812)"
4,40010,2019,1,5,719,Saturday,Austin-Forest Park,"(41.870851, -87.776812)"
...,...,...,...,...,...,...,...,...
3280,41660,2019,12,27,13898,Weekday,Lake/State,"(41.884809, -87.627813)"
3281,41660,2019,12,28,9485,Saturday,Lake/State,"(41.884809, -87.627813)"
3282,41660,2019,12,29,7581,Sunday/Holiday,Lake/State,"(41.884809, -87.627813)"
3283,41660,2019,12,30,15332,Weekday,Lake/State,"(41.884809, -87.627813)"


In [48]:
# Create a variable called filter_criteria to select the appropriate rows from the merged table so that you can sum the rides column.
filter_criteria  = ((ridership_cal_stations["month"] == 7) 
                    & (ridership_cal_stations["day_type"] == "Weekday") 
                    & (ridership_cal_stations["station_name"] == "Wilson"))

# Use .loc and the filter to select for rides
print(ridership_cal_stations.loc[filter_criteria, "rides"].sum())

140005


### Three table merge
To solidify the concept of a three DataFrame merge, practice another exercise. A reasonable extension of our review of Chicago business data would include looking at demographics information about the neighborhoods where the businesses are. A table with the median income by zip code has been provided to you. You will merge the `licenses` and `wards` tables with this new income-by-zip-code table called `zip_demo`.

In [49]:
zip_demo = pd.read_pickle("data/zip_demo.p")

In [50]:
zip_demo.head()

,zip,income
0,60630,70122
1,60640,50488
2,60622,87143
3,60614,100116
4,60608,41226


In [54]:
# Starting with the licenses table, merge to it the zip_demo table on the zip column. Then merge the resulting table to the wards table on the ward column. Save result of the three merged tables to a variable named licenses_zip_ward.
licenses_zip_ward = licenses.merge(zip_demo, on="zip").merge(wards, on="ward")
licenses_zip_ward.head()

,account,ward,aid,business,address_x,zip_x,income,alderman,address_y,zip_y
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616,46340,Pat Dowell,5046 SOUTH STATE STREET,60609
1,10,10,829,HONEYBEERS,13200 S HOUSTON AVE,60633,50164,Susan Sadlowski Garza,10500 SOUTH EWING AVENUE,60617
2,10002,14,775,CELINA DELI,5089 S ARCHER AVE,60632,42335,Edward M. Burke,2650 WEST 51ST STREET,60632
3,10005,12,NaN,KRAFT FOODS NORTH AMERICA,2005 W 43RD ST,60609,33959,George Cardenas,3476 SOUTH ARCHER AVENUE,60608
4,10044,44,638,NEYBOUR'S TAVERN & GRILLE,3651 N SOUTHPORT AVE,60613,79565,Tom Tunney,3223 NORTH SHEFFIELD AVENUE,60657


In [55]:
# Group the results of the three merged tables by the column alderman and find the median income.
display(licenses_zip_ward.groupby("alderman").agg({"income":"median"}))


,income
alderman,
Ameya Pawar,66246.0
Anthony A. Beale,38206.0
Anthony V. Napolitano,82226.0
Ariel E. Reyboras,41307.0
Brendan Reilly,110215.0
Brian Hopkins,87143.0
Carlos Ramirez-Rosa,66246.0
Carrie M. Austin,38206.0
Chris Taliaferro,55566.0


### One-to-many merge with multiple tables
In this exercise, assume that you are looking to start a business in the city of Chicago. Your perfect idea is to start a company that uses goats to mow the lawn for other businesses. However, you have to choose a location in the city to put your goat farm. You need a location with a great deal of space and relatively few businesses and people around to avoid complaints about the smell. You will need to merge three tables to help you choose your location. The `land_use` table has info on the percentage of vacant land by city ward. The `census` table has population by ward, and the `licenses` table lists businesses by ward.

In [57]:
land_use = pd.read_pickle("data/land_use.p")
land_use.head()

,ward,residential,commercial,industrial,vacant,other
0,1,41,9,2,2,46
1,2,31,11,6,2,50
2,3,20,5,3,13,59
3,4,22,13,0,7,58
4,5,25,3,1,3,68


In [58]:
census.head()

,ward,pop_2000,pop_2010,change,address,zip
0,1,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637


In [59]:
licenses.head()

,account,ward,aid,business,address,zip
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616
1,10,10,829,HONEYBEERS,13200 S HOUSTON AVE,60633
2,10002,14,775,CELINA DELI,5089 S ARCHER AVE,60632
3,10005,12,NaN,KRAFT FOODS NORTH AMERICA,2005 W 43RD ST,60609
4,10044,44,638,NEYBOUR'S TAVERN & GRILLE,3651 N SOUTHPORT AVE,60613


In [63]:
# Merge land_use and census on the ward column. Merge the result of this with licenses on the ward column, using the suffix _cen for the left table and _lic for the right table. Save this to the variable land_cen_lic.
land_cen_lic = land_use.merge(census, on="ward").merge(licenses, on="ward", suffixes=("_cen", "_lic"))
land_cen_lic.head()

,ward,residential,commercial,industrial,vacant,other,pop_2000,pop_2010,change,address_cen,zip_cen,account,aid,business,address_lic,zip_lic
0,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,12024,NaN,DIGILOG ELECTRONICS,1038 N ASHLAND AVE,60622
1,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,14446,743,EMPTY BOTTLE INC,1035 N WESTERN AVE 1ST,60622
2,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,14624,775,LITTLE MEL'S HOT DOG,2205 N CALIFORNIA AVE,60647
3,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,14987,NaN,MR. BROWN'S LOUNGE,2301 W CHICAGO AVE 1ST,60622
4,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,15642,814,Beat Kitchen,2000-2100 W DIVISION ST,60622


In [ ]:
# Group land_cen_lic by ward, pop_2010 (the population in 2010), and vacant, then count the number of accounts. Save the results to pop_vac_lic.
# The as_index=False parameter keeps the grouping columns as regular columns instead of making them the index
# The agg() function applies the count aggregation to the 'account' column
pop_vac_lic = land_cen_lic.groupby(["ward", "pop_2010", "vacant"], as_index=False).agg({"account":"count"})
pop_vac_lic.head()

,ward,pop_2010,vacant,account
0,1,56149,2,253
1,10,51535,14,130
2,11,51497,5,201
3,12,52235,4,255
4,13,53722,1,101


In [69]:
# Sort pop_vac_lic by vacant, account, an pop_2010 in descending, ascending, and ascending order respectively. Save it as sorted_pop_vac_lic.
sorted_pop_vac_lic = pop_vac_lic.sort_values(by=["vacant", "account", "pop_2010"], ascending=[False, True, True])
# Print the top few rows of sorted_pop_vac_lic
display(sorted_pop_vac_lic.head())

,ward,pop_2010,vacant,account
47,7,51581,19,80
12,20,52372,15,123
1,10,51535,14,130
16,24,54909,13,98
7,16,51954,13,156
